https://ibiblio.org/e-notes/Perc/xy.htm#:~:text=Eij%20%3D%20


In [1]:
import xygame as xg
import time as tm

%config InlineBackend.figure_formats = ['svg']
%matplotlib inline

In [2]:
start_time = tm.time()

rows = 20
cols = 20
beta = 1.1199
J = 1


In [3]:
time = 30
lattice = xg.field_generator(rows,cols)
b_field = xg.field_generator(rows,cols, left_most=0, right_most=0)
model = xg.full_model(lattice=lattice, B_field=b_field, beta=beta, time=time, J=J)

In [4]:
images = [lattice.copy()]
total_energy_list = [xg.get_net_energy(model)]
total_spin_list = [xg.get_net_spin(model)]
avg_energy_list = [xg.get_avg_energy(model)]
avg_M_list = [xg.get_avg_M(model)]
corr_func_list = []
L_t_list = []

time_steps = []

In [5]:
time = 300
E = 0
for i in range(time):
    a = xg.metropolis(model)
    if a == True:
        E +=1
    # if (time % 10 == 0):
    #     images.append(model.lattice.copy())
    #     total_energy_list.append(xg.get_net_energy(model))
    #     total_spin_list.append(xg.get_net_spin(model))
    #     avg_energy_list.append(xg.get_avg_energy(model))
    #     avg_M_list.append(xg.get_avg_M(model))    
    #     corr_func = xg.compute_correlation_function(model)
    #     corr_func_list.append(corr_func)
    #     L_t = xg.find_characteristic_length(corr_func)
        # time_steps.append(i)
        # L_t_list.append(L_t)
print(E)

# print(f"Step {time}, Correlation Function: {corr_func}")
# print(f'Characteristic length scale: {L_t}')

0.6784773326419125
0.3849111554937883
-0.06078851812275887
0.38381551268386604
-0.17667623818809275
0.4748102192802012
-0.023319683213332887
-0.11098750169615257
0.4629858914769702
-0.40326444704935716
-0.41524995360593575
0.07940853264702552
0.29914648478784045
0.1356880518919903
0.19376593047350288
-0.7264051882879631
-0.051517801675188046
0.1709869318333168
-0.39965391661336813
0.3701170399960372
-0.1617473515278327
-0.3147008745478631
0.7053745331878913
0.1441709965309934
-0.5718326938801055
0.3464144810063763
-0.47774476555664835
-0.6554262039003367
-0.3384617488310009
-0.28326028531375746
-0.2781154431152282
0.4164041617809024
-0.66584556792971
0.2809423871586505
0.06349308157898581
0.015412363011074626
0.19972891400062642
-0.9064576684976715
0.7456609327476111
0.26663104236807966
-0.8150050963432652
-0.0031609628194119166
-0.09986207159844529
-0.5768106064662073
-0.08551601364646044
-0.22201023121603614
-0.2387573100348105
-0.6417713798719673
-0.07954578263604262
-0.548515849915

In [6]:
xg.display_xy_sequence(images)

interactive(children=(IntSlider(value=0, description='frame', max=0), Output()), _dom_classes=('widget-interac…

<function xygame.display_xy_sequence.<locals>._show(frame=(0, 0))>

In [7]:
# xg.display_correlation_func_sequence(corr_func_list)

In [8]:
# time_steps = np.array(time_steps)
# L_t_list = np.array(L_t_list)

# # Fit the characteristic length data to the power law
# popt, pcov = curve_fit(xg.power_law, time_steps, L_t_list)
# b, a = popt
# print(f"Growth exponent b: {b}")

# # Plot the results
# plt.figure(figsize=(10, 6))
# plt.loglog(time_steps, L_t_list, 'o', label='Data')
# plt.loglog(time_steps, xg.power_law(time_steps, *popt), '-', label=f'Fit: $L(t) \sim t^{{{b:.2f}}}$')
# plt.xlabel('Time (t)')
# plt.ylabel('Characteristic Length (L(t))')
# plt.legend()
# plt.show()

In [9]:
# xg.graph_list(total_energy_list, "time", "energy", "Energy vs Time")
# xg.graph_list(avg_M_list, "time", "average M", "average M vs Time")

In [10]:
# xg.display_xy_sequence_html(images, folder_file_name = "my_frame/animation.html")
# xg.export_mp4(image_folder_name = "my_frame/animation_frames", output_file_name="my_animation", frame_rate=48)

In [11]:
# # first way
# defects = xg.detect_defects(lattice)

# # second way
# defects_part_2 = xg.detect_defects_part_2(lattice)

# # Plot the spin structure and the detected defects
# X, Y = np.meshgrid(np.arange(cols), np.arange(rows))
# U = np.cos(lattice)
# V = np.sin(lattice)

# plt.figure(figsize=(8, 8))
# plt.quiver(X, Y, U, V, pivot='mid')
# for (x, y) in defects:
#     plt.plot(x, y, 'ro' if x % 2 == 0 else 'bo', markersize=5)  # red for vortices, blue for antivortices with small marker
# for (x, y) in defects_part_2:
#     plt.plot(x, y, 'ko' if x % 2 == 0 else 'go', markersize=1)  # red for vortices, blue for antivortices with small marker
# plt.xlim(-1, cols)
# plt.ylim(-1, rows)
# plt.title('Spin Structure with Detected Defects')
# plt.xlabel('X-axis')
# plt.ylabel('Y-axis')
# plt.grid()
# plt.show()

In [12]:
# end_time = tm.time()
# total_time = end_time - start_time
# print(f"Total time taken to run the notebook: {total_time} seconds")
